In [12]:
from utility.utility_general import *
# from utility.util_crystal import *
# from plots import *
from time import sleep

pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', 500)
# pd.set_option('display.max_rows',None)
pd.set_option('display.max_colwidth', 500)


# from pymatgen import MPRester
from pymatgen.ext.matproj import MPRester
from ase.formula import Formula

In [2]:
crystals_df = load_var(f'{data_path}/data_bases/cod/cif_pickle/df.pkl')
sc_crystals_df = load_var(f'{data_path}/data_bases/cod/cif_supercell_pickle/df.pkl')

# Removing: broken CIFs & min_occupancy < 1
crystals_df = crystals_df[(crystals_df['minimum occupancy'] == 1) & (crystals_df['error'] == False)]
sc_crystals_df = sc_crystals_df[(sc_crystals_df['minimum occupancy'] == 1) & (sc_crystals_df['error'] == False)]
sc_crystals_df = sc_crystals_df[~sc_crystals_df['id'].isin(crystals_df['id'])] # Only adding CIFs that 

df = pd.concat([crystals_df, sc_crystals_df]).drop(['minimum occupancy', 'error'], axis=1).reset_index(drop=True)
df['filename'] = df['filename'].str.replace(local_data_path, '')
# removing min_atomic_dist <= 0.5
cod_df = df[df['min_atomic_dist'] > 0.5].reset_index(drop=True)
cod_df = cod_df[['id', 'spacegroup']]

In [13]:
thermo = load_var(f'{data_path}/thermoelectric_materials/atoms_list_top_10.pkl')
thermo = thermo.drop_duplicates(subset=['filename']).reset_index(drop=True)
assert ~thermo.duplicated(subset=['filename']).any()
thermo['sgn'] = None
thermo['HM'] = None
f_list = []
for i, _ in thermo.iterrows():
    try:
        f = Formula(thermo['formula'][i]).reduce()[0].format('hill')
    except:
        f_list.append(None)
        continue
    f_list.append(f)
thermo['formula_hill'] = f_list
# len(thermo)
thermo['id'] = None
thermo.loc[thermo['data_set'] == 'cod', 'id'] = thermo.loc[thermo['data_set'] == 'cod', 'atoms']
thermo = pd.merge(thermo, cod_df, on=['id'], how='left')

f_list = []
for i in thermo.loc[thermo['id'].notna(), 'spacegroup']:
    try:
        f = i.no
    except:
        f_list.append(None)
        continue
    f_list.append(f)
f_list
thermo.loc[thermo['id'].notna(), 'sgn'] = f_list #[i.no for i in thermo.loc[thermo['id'].notna(), 'spacegroup']]
f_list = []
for i in thermo.loc[thermo['id'].notna(), 'spacegroup']:
    try:
        f = i.symbol
    except:
        f_list.append(None)
        continue
    f_list.append(f)
f_list
thermo.loc[thermo['id'].notna(), 'HM'] = f_list #[i.no for i in thermo.loc[thermo['id'].notna(), 'spacegroup']]

# del thermo['spacegroup']
thermo.loc[thermo['data_set'] == 'mp', 'sgn'] = [i['spacegroup']['number'] for i in \
                                                 thermo[thermo['data_set'] == 'mp']['atoms']]
thermo.loc[thermo['data_set'] == 'mp', 'HM'] = [i['spacegroup']['symbol'] for i in \
                                                 thermo[thermo['data_set'] == 'mp']['atoms']]
thermo = thermo.sort_values('data_set') # COD values are being first
thermo['db'] = thermo['data_set']
# Making sure no MP is an equivalent of COD sample
ind_1 = thermo.duplicated(subset=['sgn', 'formula_hill'], keep='first')
ind_2 = thermo['db']=='mp'
ind = ind_2 & ind_1
thermo.loc[ind, 'db'] = thermo.loc[ind, 'db'] + '-avail-in-cod'

# tagging the duplicates within the dataset
ind = thermo.duplicated(subset=['data_set', 'sgn', 'formula_hill'], keep='first')
thermo.loc[ind, 'db'] = thermo.loc[ind, 'db'] + '-duplicated'
thermo = thermo.sort_index()
thermo = thermo[['year', 'formula', 'db', 'filename', 'sgn', 'HM', 'formula_hill']]
# thermo.to_csv(f'{data_path}/thermoelectric_materials/thermoelectric_materials.csv', index=False)
thermo

,year,formula,db,filename,sgn,HM,formula_hill
0,2001,EuH3Li,mp,/home/ali/Data/cod/thermoelectric/cif-top-10s/mp-541365.cif,221,Pm-3m,H3EuLi
1,2001,CdGeP2,mp-avail-in-cod,/home/ali/Data/cod/thermoelectric/cif-top-10s/mp-3668.cif,122,I-42d,CdGeP2
2,2001,CdGeP2,mp,/home/ali/Data/cod/thermoelectric/cif-top-10s/mp-16268.cif,225,Fm-3m,CdGeP2
3,2001,CdGeP2,mp,/home/ali/Data/cod/thermoelectric/cif-top-10s/mp-1226745.cif,160,R3m,CdGeP2
4,2001,CdGeP2,cod,/home/ali/Data/cod/thermoelectric/cif-top-10s/8103559.cif,122,I -4 2 d,CdGeP2
...,...,...,...,...,...,...,...
209,2018,CuTe,mp-avail-in-cod-duplicated,/home/ali/Data/cod/thermoelectric/cif-top-10s/mp-1181516.cif,59,Pmmn,CuTe
210,2018,CuTe,mp-avail-in-cod,/home/ali/Data/cod/thermoelectric/cif-top-10s/mp-20826.cif,59,Pmmn,CuTe
211,2018,CuTe,cod,/home/ali/Data/cod/thermoelectric/cif-top-10s/1526237.cif,59,P m m n,CuTe
212,2018,CuTe,cod-duplicated,/home/ali/Data/cod/thermoelectric/cif-top-10s/9011846.cif,59,P m m n,CuTe


In [10]:
thermo = pd.read_csv(f'{data_path}/thermoelectric_materials/thermoelectric_materials.csv')
thermo['db'].value_counts()

mp                            66
cod                           56
mp-avail-in-cod               43
cod-duplicated                42
mp-avail-in-cod-duplicated     7
Name: db, dtype: int64